# Modelos de redes neuronales

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Carga de datos

In [2]:
X_train, X_test, y_train, y_test = (
    pd.read_csv('data/X_train_extended.csv'),
    pd.read_csv('data/X_test_extended.csv'),
    pd.read_csv('data/y_train.csv'),
    pd.read_csv('data/y_test.csv')
)

In [3]:
X_train, X_test = (
    X_train.drop(columns=['Type_L', 'Type_M']),
    X_test.drop(columns=['Type_L', 'Type_M'])
)

y_train, y_test = y_train['Failure type'], y_test['Failure type']

Para una red neuronal, las variables objetivos las codificaremoss usando one-hot encoding.

In [4]:
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))

In [5]:
y_train_encoded[0:10, :]

array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.]])

# Definición de la red neuronal

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.utils import class_weight
import numpy as np

2024-07-26 16:18:03.476725: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 16:18:03.477608: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 16:18:03.482228: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 16:18:03.492907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 16:18:03.510141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

## Calculamos pesos para tener en cuenta el desbalance

In [7]:
# Calculate class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train.values.flatten()
)

class_weights_dict = dict(enumerate(class_weights))

## Definición del modelo y entrenamiento

In [8]:
# Define the model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(7,)),  # 7 input features
    layers.Dense(32, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 classes for the output
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['recall', 'f1_score'])

/home/carroyo/personal_python_projects/predictive-maintenance-milling/tmp/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1722032311.506935  425983 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 16:18:31.509617: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
isinstance(model, tf.keras.models.Model)

True

In [10]:
# Train the model
model.fit(
    X_train.values,
    y_train_encoded,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    shuffle=True,
    class_weight=class_weights_dict
)

Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - f1_score: 0.0389 - loss: 2.0746 - recall: 8.2227e-04 - val_f1_score: 0.1473 - val_loss: 1.6601 - val_recall: 0.0012
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.1611 - loss: 1.4330 - recall: 0.0228 - val_f1_score: 0.1793 - val_loss: 1.4137 - val_recall: 0.1231
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.1727 - loss: 1.1116 - recall: 0.1675 - val_f1_score: 0.1683 - val_loss: 1.3278 - val_recall: 0.2463
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.1846 - loss: 0.8842 - recall: 0.2983 - val_f1_score: 0.2038 - val_loss: 1.2350 - val_recall: 0.3050
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.2080 - loss: 0.8941 - recall: 0.3361 - val_f1_score: 0.1966 - val_loss: 1.1808 - val_recall: 0.3500
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.2184 - loss: 0.7844 - recall: 0.3763 - val_f1_score: 0.2327 - val_loss: 1.1843 - val_recall: 0.3444


In [13]:
# Evaluate the model on the test data
loss = model.evaluate(X_test, y_test_encoded)
# print(f'Test accuracy: {accuracy}')
loss

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3144 - loss: 0.8612 - recall: 0.5914


[0.8564415574073792,
 0.5950000286102295,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([0.7721639 , 0.06666666, 0.29921257, 0.47619042, 0.25742573,
        0.00677966, 0.37037033], dtype=float32)>]

## Predicción

In [14]:
y_pred = model.predict(X_test.values)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [15]:
# For each row in y_pred, get the index of the highest value
y_pred_argmax = np.argmax(y_pred, axis=1)

In [16]:
y_pred_argmax[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 5])

# Rendimiento del modelo

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
cr = classification_report(y_test.values, y_pred_argmax)

In [20]:
print(cr)

              precision    recall  f1-score   support

           0       1.00      0.63      0.77      1930
           1       0.03      0.89      0.07         9
           2       0.18      0.90      0.30        21
           3       0.32      0.94      0.48        16
           4       0.15      0.81      0.26        16
           5       0.00      0.33      0.01         3
           6       0.23      1.00      0.37         5

    accuracy                           0.64      2000
   macro avg       0.27      0.79      0.32      2000
weighted avg       0.97      0.64      0.76      2000



## Probamos el `NeuralNetworkWrapper`

In [21]:
from mltools import results
from mltools import models as models_tools

In [22]:
nn_wrapper = models_tools.NeuralNetworkWrapper(model=model, name='neural_network')

In [23]:
nn_wrapper.fit(X_train, y_train, sampling_strategy=None)

Type of y_train: <class 'pandas.core.series.Series'>
Shape of y_train: (8000,)
Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3289 - loss: 0.4364 - recall: 0.6239 - val_f1_score: 0.2586 - val_loss: 0.9363 - val_recall: 0.5606
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3005 - loss: 0.5159 - recall: 0.5540 - val_f1_score: 0.2974 - val_loss: 0.9450 - val_recall: 0.5437
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3345 - loss: 0.4017 - recall: 0.5942 - val_f1_score: 0.3273 - val_loss: 0.9483 - val_recall: 0.5500
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3254 - loss: 0.4040 - recall: 0.5788 - val_f1_score: 0.3412 - val_loss: 0.8214 - val_recall: 0.5894
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3364 - loss: 0.3849 - recall: 0.5891 - val_f1_score: 0.2871 - val_loss: 0.8371 - val_recall: 0.6256
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3269 - loss: 0.3625 - rec

In [24]:
nn_wrapper.evaluate(X_test, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy for neural_network: 0.6465
Classification Report for neural_network:
              precision    recall  f1-score   support

           0       1.00      0.64      0.78      1930
           1       0.06      0.89      0.11         9
           2       0.28      0.90      0.43        21
           3       0.39      0.94      0.56        16
           4       0.19      0.88      0.31        16
           5       0.00      0.33      0.00         3
           6       0.62      1.00      0.77         5

    accuracy                           0.65      2000
   macro avg       0.36      0.80      0.42      2000
weighted avg       0.97      0.65      0.77      2000


Confusion Matrix for neural_network:
[[1231  126   48   23   59  441    2]
 [   0    8    0    0    1    0    0]
 [   0    2   19    0    0    0    0]
 [   0    0    0   15    0    1    0]
 [   0    1    0    0   14    0    1]
 [   2    0    0    0    0    1    0]
 [   0    0    0    

(0.6465,
 '              precision    recall  f1-score   support\n\n           0       1.00      0.64      0.78      1930\n           1       0.06      0.89      0.11         9\n           2       0.28      0.90      0.43        21\n           3       0.39      0.94      0.56        16\n           4       0.19      0.88      0.31        16\n           5       0.00      0.33      0.00         3\n           6       0.62      1.00      0.77         5\n\n    accuracy                           0.65      2000\n   macro avg       0.36      0.80      0.42      2000\nweighted avg       0.97      0.65      0.77      2000\n')